# Deepnote에서 라이센스 솔버 활성화시키기 (Enabling licensed solvers on Deepnote)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


In [ ]:
import os
assert "DEEPNOTE_PROJECT_ID" in os.environ, "This tutorial is meant to be run on Deepnote"

Drake의 MathematicalProgram 인터페이스는 일부 상용 솔버를 지원하며, 이러한 솔버를 활성화하려면 사용자가 라이센스 파일을 제공해야 한다. 이 튜토리얼은 Deepnote에서 이 작업을 수행하는 방법에 대한 예시를 제공한다. 워크플로우를 단순화하면서도 실수로 라이센스 파일을 공유할 가능성을 최소화하는 것을 목표로 한다.

활성화가 올바르게 작동하는지 확인하기 위해서 **이 notebook을 복제**, 라이센스 파일 업로드 및 코드 실행해보자.

## Mosek

* Mosek 라이센스가 없다면 라이센스를 얻는 방법을 알아보자. 학생인 경우 [무료 개인 학생 라이센스](https://www.mosek.com/products/academic-licenses/)를 다운로드받을 수 있다. 
* 이 cell에서 여러분의 로컬 컴퓨터로부터 `mosek.lic` 파일을 업로드하도록 안내합니다.

**주 의! 절대 라이센스 파일을 공개하지 마세요!**

라이센스 파일 공유를 방지하기 위해 이 파일을 `/tmp/mosek.lic`으로 업로드하자. 매번 세션 종료 시 자동 삭제된다.

참고: 업로드한 라이센스로 협력자와 활성 세션을 공유하고자 한다면, 상대방이 `/tmp` 폴더에서 파일을 복사하면 해당 라이센스를 얻을 수 있다.

In [ ]:
import os
import os.path

import ipywidgets as widgets  # Our use requires ipywidgets >= 7.5.0
from IPython.display import display

if "MOSEKLM_LICENSE_FILE" not in os.environ:
    # If a mosek.lic file has already been uploaded, then simply use it here.
    if os.path.exists('/tmp/mosek.lic'):
        os.environ["MOSEKLM_LICENSE_FILE"] = "/tmp/mosek.lic"
    else:
        uploader = widgets.FileUpload(accept='.lic', multiple=False)
        display(uploader)

In [ ]:
if "MOSEKLM_LICENSE_FILE" not in os.environ:
    assert len(uploader.value.values()) > 0, "Please upload a license file using the Upload widget above."
    with open('/tmp/mosek.lic', 'wb') as output_file:
        output_file.write(list(uploader.value.values())[-1]['content']) 
    os.environ["MOSEKLM_LICENSE_FILE"] = "/tmp/mosek.lic"

Drake가 이제 Mosek을 사용할 수 있다고 판단한다.:

In [ ]:
from pydrake.solvers import MosekSolver

print(MosekSolver().enabled())

이제 간단한 semidefinite program을 풀어보자. 

$$\begin{aligned} \min_S \quad & \text{Trace}(S) \\ \text{subject to} \quad & S_{1,0} = 1, \\ & S \succeq 0. \end{aligned}$$

알고 있는 최적화 솔루션은  $ S = \begin{bmatrix} 1 & 1 \\ 1 & 1\end{bmatrix}.$

In [ ]:
from pydrake.solvers import MathematicalProgram, MosekSolver

prog = MathematicalProgram()
S = prog.NewSymmetricContinuousVariables(2, "S")

# S ≽ 0.
prog.AddPositiveSemidefiniteConstraint(S)

# S(1, 0) = 1
prog.AddBoundingBoxConstraint(1, 1, S[1, 0])

# min Trace(S)
prog.AddLinearCost(S[0,0] + S[1,1])

mosek = MosekSolver()
result = mosek.Solve(prog)
assert result.is_success()

print(result.GetSolution(S))

## Gurobi

Drake 바이너리 배포판에는 아직 Gurobi 솔버가 포함되어 있지 않다. 개발팀은 추후 지원을 위해 노력하고 있으며, 관련 이슈 페이지 링크 : https://github.com/RobotLocomotion/drake/issues/10804

## Clean up

만일을 위해 이제 남은 임시 라이센스 파일을 모두 삭제하도록 하자.

In [ ]:
if os.path.exists('/tmp/mosek.lic'):
    os.remove('/tmp/mosek.lic')